In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import os
import pickle
from collections import OrderedDict

import vdmlab as vdm

from loading_data import get_data
from utils_maze import get_xyedges, find_zones
from plot_decode import get_combined, compare_lengths, get_proportions
from utils_plotting import plot_decoded_compare

In [ ]:
import info.test as info

In [ ]:
data_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/data'
pickle_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/pickled'

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)
xedges, yedges = get_xyedges(position)

In [ ]:
plt.plot(position.x, position.y, 'b.', ms=2)
plt.show()

In [ ]:
from analyze_tuning_curves import analyze
tuning_curve = analyze(info)

In [ ]:
# tuning_curve_filename = 'R068d1' + '_tuning-curve.pkl'
# pickled_tuning_curve = os.path.join(pickle_filepath, tuning_curve_filename)
# with open(pickled_tuning_curve, 'rb') as fileobj:
#     tuning_curve = pickle.load(fileobj)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)

for ii in range(5):
    print(ii)
    pp = plt.pcolormesh(xx, yy, tuning_curve[ii], cmap='pink_r')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
from analyze_decode import analyze
experiment_times = ['phase3']
for experiment_time in experiment_times:
    analyze(info, tuning_curve, experiment_time, min_length=2)

In [ ]:
def compare_test(infos, combined_zones, timebin=0.025):
    lengths = dict(u=[], shortcut=[], novel=[])
    for info in infos:
        lengths['u'].append(LineString(info.u_trajectory).length)
        lengths['shortcut'].append(LineString(info.shortcut_trajectory).length)
        lengths['novel'].append(LineString(info.novel_trajectory).length)

    by_track_length = []

    for i, session in enumerate(combined_zones):
        normalized = OrderedDict()
        for exp in combined_zones[0].keys():
            normalized[exp] = dict()

        for exp in session.keys():
            for trajectory in session[exp].keys():
                normalized[exp][trajectory] = (session[exp][trajectory].n_samples * timebin)

        by_track_length.append(normalized)

    return by_track_length

In [ ]:
infos = [info]

experiment_times = ['phase3']
sessions = get_combined(infos, experiment_times)
norm_length = compare_test(infos, sessions)
proportion = get_proportions(norm_length)

plot_decoded_compare(proportion, ylabel='Proportion of time relative to track length')

In [ ]:
experiment_time = experiment_times[0]
filename = '_decode-' + experiment_time + '.pkl'
decode_filename = info.session_id + filename
pickled_decoded = os.path.join(pickle_filepath, decode_filename)

if os.path.isfile(pickled_decoded):
    with open(pickled_decoded, 'rb') as fileobj:
        decoded = pickle.load(fileobj)

In [ ]:
plt.plot(position.x, position.y, 'y.', ms=1)
plt.plot(decoded['decoded'].x, decoded['decoded'].y, 'b.', ms=2)
plt.show()

In [ ]:
decoded['decoded'].n_samples

In [ ]:
for neuron in spikes:
    print(len(neuron.time))

In [ ]:
zone = find_zones(info, remove_feeder=True)
for neuron in spikes:
    plt.plot(position.x, position.y, 'y.', ms=1)
    for spike in neuron.time:
        idx = vdm.find_nearest_idx(position.time, spike)
        point = Point([position.x[idx], position.y[idx]])
        if zone['u'].contains(point):
            plt.plot(point.xy[0], point.xy[1], 'b.', ms=2)
        if zone['shortcut'].contains(point):
            plt.plot(point.xy[0], point.xy[1], 'g.', ms=2)
        if zone['novel'].contains(point):
            plt.plot(point.xy[0], point.xy[1], 'r.', ms=2)
    plt.show()

In [ ]:
print(len(spikes))

In [ ]:
print(len(spikes[1].time))

In [ ]:
# keep_idx = np.array([1, 3, 5, 7, 9])
# spikes = np.array(spikes)
# filtered_spikes = spikes[keep_idx]

In [ ]:
# zone = find_zones(info, remove_feeder=True)
# 
# def get_n_spikes(spikes, zone, track):
#     for i, neuron in enumerate(spikes):
#         label = neuron.label
#         trajectory = []
#         for spike in neuron.time:
#             idx = vdm.find_nearest_idx(position.time, spike)
#             point = Point([position.x[idx], position.y[idx]])
#             if zone[track].contains(point):
#                 trajectory.append(spike)
#         print(track, i, ':', len(trajectory))
    
# get_n_spikes(spikes, zone, track='u')
# get_n_spikes(spikes, zone, track='shortcut')
# get_n_spikes(spikes, zone, track='novel')

In [ ]:
n_spikes = 100
zone = find_zones(info, remove_feeder=True)

new_spikes = []
for neuron in [spikes[1]]:
    label = neuron.label
    trajectory = []
    for spike in neuron.time:
        idx = vdm.find_nearest_idx(position.time, spike)
        point = Point([position.x[idx], position.y[idx]])
        if zone['u'].contains(point):
            trajectory.append(spike)
    time = trajectory[:n_spikes]
    new_spikes.append(vdm.SpikeTrain(time, label))
    
for neuron in [spikes[17]]:
    label = neuron.label
    trajectory = []
    for spike in neuron.time:
        idx = vdm.find_nearest_idx(position.time, spike)
        point = Point([position.x[idx], position.y[idx]])
        if zone['novel'].contains(point):
            trajectory.append(spike)
    time = trajectory[:n_spikes]
    new_spikes.append(vdm.SpikeTrain(time, label))
    
for neuron in [spikes[11], spikes[13], spikes[27], spikes[34], spikes[49]]:
    label = neuron.label
    trajectory = []
    for spike in neuron.time:
        idx = vdm.find_nearest_idx(position.time, spike)
        point = Point([position.x[idx], position.y[idx]])
        if zone['shortcut'].contains(point):
            trajectory.append(spike)
    time = trajectory[:n_spikes]
    new_spikes.append(vdm.SpikeTrain(time, label))

In [ ]:
for neuron in new_spikes:
    print(len(neuron.time))
len(new_spikes)

In [ ]:
filename = 'test-spike.pkl'
pickled_path = os.path.join(data_filepath, filename)
with open(pickled_path, 'wb') as fileobj:
    pickle.dump(new_spikes, fileobj)